#Reading from source CSV File.

# Define Ingestion Configuration

In [0]:

INGESTION_CONFIG = [
    {
        "source": "crm",
        "path": "/Volumes/workspace/bronze/source_system/source_crm/cust_info.csv",
        "table": "crm_cust_info"
    },
    {
        "source": "crm",
        "path": "/Volumes/workspace/bronze/source_system/source_crm/prd_info.csv",
        "table": "crm_prd_info"
    },
    {
        "source": "crm",
        "path": "/Volumes/workspace/bronze/source_system/source_crm/sales_details.csv",
        "table": "crm_sales_details"
    },
    {
        "source": "erp",
        "path": "/Volumes/workspace/bronze/source_system/source_erp/CUST_AZ12.csv",
        "table": "erp_cust_az12"
    },
    {
        "source": "erp",
        "path": "/Volumes/workspace/bronze/source_system/source_erp/LOC_A101.csv",
        "table": "erp_loc_a101"
    },
    {
        "source": "erp",
        "path": "/Volumes/workspace/bronze/source_system/source_erp/PX_CAT_G1V2.csv",
        "table": "erp_px_cat_g1v2"
    }
]

# Ingest Files into Bronze Tables

## I write columns as string data type. Cast in silver layer.


In [0]:
import pyspark.sql.functions as F

for item in INGESTION_CONFIG:
    print(f"Ingesting {item['source']} → workspace.bronze.{item['table']}")

    df = (
        spark.read
            .option("header", "true")
            .option("mode", "PERMISSIVE")
            .option("columnNameOfCorruptRecord", "_corrupt_record")
            .csv(item["path"])
            
            .withColumn("source", F.lit(item["source"]))
            .withColumn("source_file", F.col("_metadata.file_path"))
            .withColumn("ingest_timestamp", F.current_timestamp())
)

    (
        df.write
          .format("delta")
          .mode("append")
          .saveAsTable(f"workspace.bronze.{item['table']}")
    )


In [0]:
%sql
DROP TABLE IF EXISTS workspace.bronze.crm_cust_info;
DROP TABLE IF EXISTS workspace.bronze.crm_prd_info;
DROP TABLE IF EXISTS workspace.bronze.crm_sales_details;
DROP TABLE IF EXISTS workspace.bronze.erp_cust_az12;
DROP TABLE IF EXISTS workspace.bronze.erp_loc_a101;
DROP TABLE IF EXISTS workspace.bronze.erp_px_cat_g1v2;



DROP TABLE IF EXISTS workspace.silver.crm_customers;


In [0]:
%sql

SELECT * FROM workspace.bronze.crm_cust_info;